In [44]:
import pandas as pd
movies3 = pd.read_csv("../data-files/total_tmdbmovielist.csv", index_col=0, lineterminator= '\n')

/Users/parkseonghun/opt/miniconda3/envs/kdigital-busan/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
movies3 = movies3[movies3['id'].notna()] # id na아닌거 남기기

In [46]:
# 안쓰는 컬럼 제거 ( backdrop_path, belongs_to_collection, homepages, video, imdb_id, production_companies,spoken_languages, tagline,status)
movies3 =movies3.drop(columns=['backdrop_path', 'belongs_to_collection','homepage','video','imdb_id', 'production_companies','spoken_languages','tagline','status'])

In [47]:
movies3 = movies3[movies3['title'].notna()]

In [48]:
movies3 = movies3[movies3['vote_count'].notna()]

In [49]:
movies3 = movies3.drop(columns=['genres','original_language','original_title','overview','popularity','poster_path','release_date','title','vote_average','vote_count'])

In [50]:
# 1. 영화테이블에서 장르id만 추출하여 genre_ids칼럼을 만들기  

# str자료형을 list로 변경하는 함수
error_list_x = []
def genres_str_to_list(genres_str):
    import json
    # print(genres_str)
    genre_id_list = []
    try :
        genres_list = json.loads(genres_str.replace("\'", "\""))
        for production_country in genres_list:
            genre_id_list.append(production_country['name'])

    except:
        error_list_x.append(genres_str)
    
    return genre_id_list

# 장르 id를 추출하여 genre_ids칼럼 추가
movies3['production_countries'] = movies3['production_countries'].map(genres_str_to_list)
movies3.tail(5)

,adult,budget,id,production_countries,revenue,runtime
652710,False,0.0,699995.0,[Czechoslovakia],0.0,0.0
652711,False,0.0,699996.0,[],0.0,107.0
652712,False,0.0,699997.0,[],0.0,12.0
652713,False,0.0,699998.0,"[Mexico, United States of America]",0.0,0.0
652714,False,0.0,699999.0,[],0.0,47.0


In [51]:
movies3['runtime'] = movies3['runtime'].apply(lambda ov : ov if str(ov) != 'nan' else 0 )

In [52]:
movies4 = movies3.drop(columns=['production_countries'])

In [53]:
movies4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652612 entries, 0 to 652714
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   adult    652612 non-null  object 
 1   budget   652612 non-null  object 
 2   id       652612 non-null  object 
 3   revenue  652612 non-null  object 
 4   runtime  652612 non-null  float64
dtypes: float64(1), object(4)
memory usage: 29.9+ MB


In [55]:

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()
cursor.execute('DELETE FROM movie2')

error_list2 = []

movies3['runtime'] = movies3['runtime'].apply(lambda ov : ov if str(ov) != 'nan' else 0 )
for movie in movies4.values:

    sql= "insert into movie2 (adult, budget, movie_id, revenue, runtime ) values (%s, %s, %s, %s, %s)"
    try:
        cursor.execute(sql, list(movie)   )
    except:
        error_list2.append(movie)

conn.commit()
cursor.close()

conn.close()

In [56]:
len(error_list2)

125048